In [1]:
import pandas as pd
from skimage.segmentation import slic,quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from skimage.future import graph
import networkx
import glob

In [2]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [3]:
filelist=get_filelist("/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/")

In [8]:
def preprocessing(filelist):
    for i, fname in enumerate(filelist):
        image = img_as_float(io.imread(fname))
        segments = slic(image, n_segments = 50, sigma = 5)
        rag = graph.rag_mean_color(image, segments)
        f = open("graph_text_files/"+fname.split("/")[6].split(".")[0]+".txt", "a")
        for line in networkx.generate_edgelist(rag, data=False):
            f.write(line)
            f.write("\n")
        f.close()

In [9]:
preprocessing(filelist)

In [10]:
def get_text_filelist(txt_loc):
    filelist = glob.glob(txt_loc+'*.txt')
    filelist.sort()
    return filelist

In [11]:
txt_filelist=get_text_filelist("/home/sirzechlucifer/Work/CVPR_graphs/graph_text_files/")

In [12]:
len(txt_filelist)

10015

In [13]:
import os

In [14]:
def graph_preprocessing(filelist):
    for i, fname in enumerate(filelist):
        os.system("python /home/sirzechlucifer/Work/CVPR_graphs/HARP/src/harp.py --input "+fname+" --format edgelist  --model line --output /home/sirzechlucifer/Work/CVPR_graphs/harp_numpy_files/"+fname.split("/")[6].split(".")[0]+".npy"+" --sfdp-path /home/sirzechlucifer/Work/CVPR_graphs/HARP/bin/sfdp_linux")

In [15]:
graph_preprocessing(txt_filelist)

In [16]:
def get_npy_filelist(npy_loc):
    filelist = glob.glob(npy_loc+'*.npy')
    filelist.sort()
    return filelist

In [17]:
npy_filelist=get_npy_filelist("/home/sirzechlucifer/Work/CVPR_graphs/harp_numpy_files/")

In [21]:
embed=np.load(npy_filelist[0])

In [23]:
embed_flat=embed.flatten()

In [25]:
embed_flat.shape

(3072,)

In [30]:
def npy_preprocessing(filelist):
    X_orig = np.empty([10015, 3200])
    for i, fname in enumerate(filelist):
        embed=np.load(fname)
        embed_flat=embed.flatten()
        if embed_flat.shape[0]!=3200:
            result = np.zeros((3200))
            result[:embed_flat.shape[0]] = embed_flat
            X_orig[i]=result
        else:
            X_orig[i]=embed_flat

    return X_orig

In [31]:
embed_features=npy_preprocessing(npy_filelist)

In [34]:
np.save('graph_embedding.npy', embed_features)